In [3]:

from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

def create_analysis_link(link):
    new = 'https://finance.naver.com' + link
    return 'https://finance.naver.com' + link[:6] + 'coinfo' + link[10:]


links = [] # 최종적인 링크들을 담을 테이블
# 시가총액 1~200위 페이지
for page in range(1,33):
    html = urlopen('https://finance.naver.com/sise/sise_market_sum.nhn?sosok=1&page='+str(page))
    bs = BeautifulSoup(html, 'html.parser')
    rankings = bs.findAll('a',{'href' : re.compile('^(/item/main.)')})

    # 각 종목별 페이지로 연결
    for ranking in rankings:
            if 'href' in ranking.attrs:
                midLink = str(create_analysis_link(ranking.attrs['href']))
                
                # 종목별 페이지 속의 '재무분석' 페이지로 연결
                html = urlopen("{}".format(midLink))
                bs = BeautifulSoup(html,'html.parser')
                link = bs.find('div', {'class' : 'section inner_sub'}).find('iframe')
                
                if 'src' in link.attrs:
                    financialLink = link.attrs['src']
                    links.append(financialLink)
                    


from selenium import webdriver
from selenium.webdriver.common.by import By

import time

data = []

for one in links:
    # headless 설정
    options = webdriver.ChromeOptions()
    options.add_argument('headless')

    # 브라우저 생성
    browser = webdriver.Chrome(chrome_options=options)
    browser.get('{}'.format(one))
    
    try: # ETF를 걸러줄 장치
    
        # 종목 이름 구하기
        elementXpath = '//*[@id="pArea"]/div[1]/div/table/tbody/tr[1]/td/dl/dt[1]/span'
        title = browser.find_element(by=By.XPATH, value = elementXpath).text

        # 시가총액 구하기
        elementXpath = '//*[@summary="기업의 기본적인 시세정보(주가/전일대비/수익률,52주최고/최저,액면가,거래량/거래대금,시가총액,유동주식비율,외국인지분율,52주베타,수익률(1M/3M/6M/1Y))를 제공합니다."]/tbody/tr[5]/td'
        market_cap = float(browser.find_element(by=By.XPATH, value = elementXpath).text[:-2].replace(",",""))

        # 포괄손익계산서 이동하기
        browser.find_element_by_link_text("재무분석").click()
        browser.find_element_by_link_text("포괄손익계산서").click()

        time.sleep(1)
        # 영업이익 비교하기_ 조건 불합시 continue
        try: 
            elementXpath = '//table[@summary="IFRS연결 연간 재무 정보를 제공합니다."][2]/tbody/tr[58]/td[5]'
            O_profit2020 = browser.find_element(by=By.XPATH, value = elementXpath).text

            elementXpath = '//table[@summary="IFRS연결 연간 재무 정보를 제공합니다."][2]/tbody/tr[58]/td[4]'
            O_profit2019 = browser.find_element(by=By.XPATH, value = elementXpath).text

            if O_profit2020 <= O_profit2019:
                browser.quit()
                continue

        except: # 영업이익이 안뜨는 상황.
            browser.quit()
            continue

        # 재무상태표 이동하기
        browser.find_element_by_link_text("재무상태표").click()
        time.sleep(1)

        # 순운전자본 구하기
        elementXpath = '//table[@summary="IFRS연결 연간 재무 정보를 제공합니다."][2]/tbody/tr[2]/td[5]'
        current_asset = float(browser.find_element(by=By.XPATH, value = elementXpath).text.replace(",",""))

        elementXpath = '//table[@summary="IFRS연결 연간 재무 정보를 제공합니다."][2]/tbody/tr[128]/td[5]'
        current_liabilities = float(browser.find_element(by=By.XPATH, value = elementXpath).text.replace(",",""))

        elementXpath = '//table[@summary="IFRS연결 연간 재무 정보를 제공합니다."][2]/tbody/tr[47]/td[5]'
        cashable_asset = float(browser.find_element(by=By.XPATH, value = elementXpath).text.replace(",",""))

        net_working_capital = current_asset - current_liabilities + cashable_asset

        # 순운전자본과 시가총액 비교
        if net_working_capital < market_cap:
            browser.quit()
            continue

        # 조건으로부터 살아남았다면, 수익률 계산(추가 작성 예정)

        # 조건으로부터 살아남았다면, 리스트 테이블에 추가
        data.append((title, current_asset, current_liabilities, net_working_capital, market_cap, str(round(market_cap/net_working_capital*100,2))+'%' ))
                    
    except: # ETF는 거른다.
        continue

# 데이터 저장
import csv

csvFile = open('COSDAC200_Finance.csv', 'wt+', newline = '')
writer = csv.writer(csvFile)

writer.writerow(('기업 이름', '유동자산(억)', '유동부채(억)', '순운전자본(억)', '시가총액(억)', '시총/순운전자본'))

try:
    for one in data:
        writer.writerow(one)
        
finally:
    csvFile.close()

C:\Users\dck\AppData\Local\Temp/ipykernel_19472/2607515557.py:46: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(chrome_options=options)
C:\Users\dck\AppData\Local\Temp/ipykernel_19472/2607515557.py:60: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  browser.find_element_by_link_text("재무분석").click()
C:\Users\dck\AppData\Local\Temp/ipykernel_19472/2607515557.py:61: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  browser.find_element_by_link_text("포괄손익계산서").click()
C:\Users\dck\AppData\Local\Temp/ipykernel_19472/2607515557.py:81: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  browser.find_element_by_link_text("재무상태표").click()


In [ ]:
"""
시간이 없어 아직 테스트도 못해봤지만
데이터가 잘 정리되어 저장되었다고 가정하자.

이번에는 거르는 데이터가 많아 훨씬 빠르게 동작할 것이다.


# 개선점(1)
수익률을 내가 일일이 찾아볼 수는 없는 일이다.
내일 다시 작업을 한다면 수익률까지 추출할 수 있는 방법을 찾아내자. 어렵지 않을 것이다.

# 개선점(2)
페이지가 아직 4페이지 까지로, 200위까지밖에 반영하지 못한다.
한 10페이지까지 해버리자. 동작만 된다면 못할게 뭔가. 
메모리에 누적되는 방식도 아니므로 20페이지도 무리없다.
대신 이정도 사이즈라면 내가 수익률을 찾아보는 것은 당연히 무리이므로 반드시 수익률까지 시간 걸리지 않게 구해야한다.

# 개선점(3)
이것은 어디까지나 '테스트'용이다.
1년동안의 수익률을 살펴보는 정도.
결과가 좋다면 앞으로 1년의 내용을 작성해보자.
실제로 투자로 이어질 수 있을 정도까지 간다면 좋겠다.
"""